In [10]:
import os
import time
import pandas as pd

from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout



Notebook for Scrapping Necessary Data. 

Put this in a function for easy re-running. Also, move data to csv after scrapping so that the only reruns need to be the current season games.

In [77]:
SEASONS = list(range(2016,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [142]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                # print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

Might need to just run one season at a time. I think I am getting shut out from basketball reference for too many scrapes...

In [116]:
SEASONS = []

for season in SEASONS:
    await scrape_season(season)

In [113]:
standings_files = os.listdir(STANDINGS_DIR)

In [139]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#content")
        if not html:
                continue
        with open(save_path, "w+") as f:
                f.write(html)

Only going to do 2022-23 season because we have the pre-made csv from 2016-2022 via GitHub

In [140]:
standings_files = [s for s in standings_files if ".html" and "2023" in s]
standings_files

['NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-january.html',
 'NBA_2023_games-november.html',
 'NBA_2023_games-december.html',
 'NBA_2023_games-february.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html']

In [143]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

Timeout error on https://www.basketball-reference.com/boxscores/202304070CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202304140MIN.html
Timeout error on https://www.basketball-reference.com/boxscores/202211020MIA.html
